In [1]:
import sys, os, platform
import h5py, tarfile
import numpy as np
from pandas import Series, DataFrame

In [29]:
h5filepath = '../out/RFrames_SH510050.h5'
# h5filepath = '../out/RFrames_SZ159948.h5'
h5file = h5py.File(h5filepath, 'r')
h5file.keys()

<KeysViewHDF5 ['ReplayFrame:DQN_Cnn1Dx4.1556_3.F000001', 'ReplayFrame:DQN_Cnn1Dx4.1556_3.F000002', 'ReplayFrame:DQN_Cnn1Dx4.1556_3.F000003', 'ReplayFrame:DQN_Cnn1Dx4.1556_3.F000004', 'ReplayFrame:DQN_Cnn1Dx4.1556_3.F000005', 'ReplayFrame:DQN_Cnn1Dx4.1556_3.F000006', 'ReplayFrame:DQN_Cnn1Dx4.1556_3.F000007', 'ReplayFrame:DQN_Cnn1Dx4.1556_3.F000008', 'ReplayFrame:DQN_Cnn1Dx4.1556_3.F000009', 'ReplayFrame:DQN_Cnn1Dx4.1556_3.F000010', 'ReplayFrame:DQN_Cnn1Dx4.1556_3.F000011', 'ReplayFrame:DQN_Cnn1Dx4.1556_3.F000012', 'ReplayFrame:DQN_Cnn1Dx4.1556_3.F000013', 'ReplayFrame:DQN_Cnn1Dx4.1556_3.F000014', 'ReplayFrame:DQN_Cnn1Dx4.1556_3.F000015', 'ReplayFrame:DQN_Cnn1Dx4.1556_3.F000016', 'ReplayFrame:DQN_Cnn1Dx4.1556_3.F000017', 'ReplayFrame:DQN_Cnn1Dx4.1556_3.F000018', 'ReplayFrame:DQN_Cnn1Dx4.1556_3.F000019', 'ReplayFrame:DQN_Cnn1Dx4.1556_3.F000020', 'ReplayFrame:DQN_Cnn1Dx4.1556_3.F000021', 'ReplayFrame:DQN_Cnn1Dx4.1556_3.F000022', 'ReplayFrame:DQN_Cnn1Dx4.1556_3.F000023', 'ReplayFrame:DQN_Cn

In [30]:
framesInHd5 = []
for name in h5file.keys():
    if 'ReplayFrame:' == name[:len('ReplayFrame:')] :
        framesInHd5.append(name)

if len(framesInHd5)>3:
    del framesInHd5[0]
    del framesInHd5[-1]

framesInHd5

['ReplayFrame:DQN_Cnn1Dx4.1556_3.F000002',
 'ReplayFrame:DQN_Cnn1Dx4.1556_3.F000003',
 'ReplayFrame:DQN_Cnn1Dx4.1556_3.F000004',
 'ReplayFrame:DQN_Cnn1Dx4.1556_3.F000005',
 'ReplayFrame:DQN_Cnn1Dx4.1556_3.F000006',
 'ReplayFrame:DQN_Cnn1Dx4.1556_3.F000007',
 'ReplayFrame:DQN_Cnn1Dx4.1556_3.F000008',
 'ReplayFrame:DQN_Cnn1Dx4.1556_3.F000009',
 'ReplayFrame:DQN_Cnn1Dx4.1556_3.F000010',
 'ReplayFrame:DQN_Cnn1Dx4.1556_3.F000011',
 'ReplayFrame:DQN_Cnn1Dx4.1556_3.F000012',
 'ReplayFrame:DQN_Cnn1Dx4.1556_3.F000013',
 'ReplayFrame:DQN_Cnn1Dx4.1556_3.F000014',
 'ReplayFrame:DQN_Cnn1Dx4.1556_3.F000015',
 'ReplayFrame:DQN_Cnn1Dx4.1556_3.F000016',
 'ReplayFrame:DQN_Cnn1Dx4.1556_3.F000017',
 'ReplayFrame:DQN_Cnn1Dx4.1556_3.F000018',
 'ReplayFrame:DQN_Cnn1Dx4.1556_3.F000019',
 'ReplayFrame:DQN_Cnn1Dx4.1556_3.F000020',
 'ReplayFrame:DQN_Cnn1Dx4.1556_3.F000021',
 'ReplayFrame:DQN_Cnn1Dx4.1556_3.F000022',
 'ReplayFrame:DQN_Cnn1Dx4.1556_3.F000023',
 'ReplayFrame:DQN_Cnn1Dx4.1556_3.F000024',
 'ReplayFra

In [31]:
fname = framesInHd5[10]
fname

'ReplayFrame:DQN_Cnn1Dx4.1556_3.F000012'

In [77]:
H5_COLS = 'state,action,reward,next_state,done'
samplePool = {
            'state':[],
            'action':[],
            'reward':[],
            'next_state':[],
            'done':[],
        }

frame = h5file[fname]
statebatch = np.array(frame['state'].value)
actionbatch = np.array(frame['action'].value)
rewardbatch = np.array(frame['reward'].value)
#for col in H5_COLS.split(',') :
#    incrematal = list(frame[col].value)
#    samplePerFrame = len(incrematal)
#    samplePool[col] += incrematal
statebatch.shape

C:\Anaconda3\lib\site-packages\h5py\_hl\dataset.py:313: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  "Use dataset[()] instead.", H5pyDeprecationWarning)


(8192, 1556)

In [48]:
statebatch.shape[1]

389

In [90]:
d2 = int(1556 /4)
statebatch = statebatch.reshape(statebatch.shape[0],d2,4)
state = statebatch[5]
header = state[:3]
pos = state[1:2]
time = state[2:3]
KL1m = state[3:(3+30)]
KL5m = state[33:(33+96)]
KL1d = state[(33+96):]
state

array([[1.3555834e+05, 1.3555834e+05, 0.0000000e+00, 0.0000000e+00],
       [1.9560000e+00, 1.8455826e+00, 0.0000000e+00, 0.0000000e+00],
       [1.0000000e+01, 1.7000000e+01, 1.0000000e+00, 8.3300000e+02],
       ...,
       [1.1069791e+00, 5.1916583e+02, 1.1105450e+00, 1.1049414e+00],
       [1.1079980e+00, 9.8924194e+02, 1.1136017e+00, 1.1039226e+00],
       [1.0988283e+00, 2.2339622e+03, 1.1059602e+00, 1.0927153e+00]],
      dtype=float32)

In [91]:
DataFrame(header,columns=['cashAail','cashTotal','posVal','L'])

,cashAail,cashTotal,posVal,L
0,135558.34375,135558.343750,0.0,0.0
1,1.95600,1.845583,0.0,0.0
2,10.00000,17.000000,1.0,833.0


In [85]:
DataFrame(KL1m,columns=['P','V','H','L'])

,P,V,H,L
0,0.996434,0.662032,0.996434,0.996434
1,1.004585,0.000000,1.004585,1.004585
2,1.004585,2.648130,1.004585,1.004585
3,1.006113,0.000000,1.006113,1.006113
4,1.006113,8.275406,1.006113,1.006113
5,0.996434,1.688183,1.001528,0.996434
6,1.001528,1.489573,1.001528,1.001528
7,1.002547,0.000000,1.002547,1.002547
8,1.002547,0.000000,1.002547,1.002547
9,1.002547,0.000000,1.002547,1.002547


In [89]:
DataFrame(KL5m,columns=['P','V','H','L'])

,P,V,H,L
0,1.006113,11.453161,1.006113,0.996434
1,1.002547,0.000000,1.002547,1.002547
2,1.002547,0.000000,1.002547,1.002547
3,1.002547,0.099305,1.002547,1.002547
4,1.003566,0.000000,1.003566,1.003566
5,1.003566,0.000000,1.003566,1.003566
6,1.003566,6.620325,1.003566,1.003566
7,1.003566,15.855677,1.003566,1.001019
8,1.001019,0.000000,1.001019,1.001019
9,1.001019,0.000000,1.001019,1.001019


In [88]:
DataFrame(KL1d,columns=['P','V','H','L'])

,P,V,H,L
0,1.000000,100.000000,1.018849,1.000000
1,1.018849,164.382660,1.018849,1.007132
2,1.007132,340.516388,1.019358,1.004585
3,1.006113,645.018188,1.018849,1.005094
4,1.014264,164.051636,1.014264,1.000000
5,1.001019,50.645481,1.006113,0.992868
6,0.987774,48.262165,0.988283,0.983698
7,0.983698,210.592514,0.987264,0.977585
8,0.977585,58.258854,0.981151,0.970453
9,0.970453,122.442902,0.982680,0.964340


In [82]:
actionseq = ['H' if i[0]>0.5 else 'B' if i[1]>0.5 else 'S' for i in actionbatch ]
actionseq


['S',
 'S',
 'S',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'H',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S'